In [ ]:
from sentence_transformers import SentenceTransformer
import json
import os
import requests

/Users/santiago/Desktop/Ab-Inbev/upskilling/llm-zoomcamp/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../docs/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
docs_raw

[{'id': 'a1b2c3d4-e5f6-7a8b-9c0d-e1f2a3b4c5d6',
  'user_id': 'whatsapp:+573001234567',
  'raw_text': 'Cita: Reunión con finanzas el 15/07/2025 a las 10:00 en la oficina central',
  'title': 'Reunión con finanzas',
  'datetime': '2025-07-15T10:00:00',
  'notes': 'Oficina central'},
 {'id': 'b2c3d4e5-f6a7-8b9c-0d1e-f2a3b4c5d6e7',
  'user_id': 'whatsapp:+573001234567',
  'raw_text': 'Doctor: Control anual el 18/07/2025 a las 08:30 con el Dr. Ramírez, llevar resultados de laboratorio',
  'title': 'Control anual con Dr. Ramírez',
  'datetime': '2025-07-18T08:30:00',
  'notes': 'Llevar resultados de laboratorio'},
 {'id': 'c3d4e5f6-a7b8-9c0d-1e2f-a3b4c5d6e7f8',
  'user_id': 'whatsapp:+573001234567',
  'raw_text': 'Cita: Entrega de proyecto el 20/07/2025 a las 17:00 vía Google Meet',
  'title': 'Entrega de proyecto',
  'datetime': '2025-07-20T17:00:00',
  'notes': 'Vía Google Meet'},
 {'id': 'd4e5f6a7-b8c9-0d1e-2f3a-b4c5d6e7f8a9',
  'user_id': 'whatsapp:+573001234567',
  'raw_text': "Película

In [ ]:
model = SentenceTransformer("all-mpnet-base-v2")

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
Eres un asistente personal de calendario para la familia Aguirre Rojas. Responde a la CONSULTA usando únicamente la información de las CITAS proporcionadas.

CONSULTA:
{question}

CITAS:
{context}

RESPUESTA:
""".strip()

    # Construimos el contexto listando cada cita con fecha, título y notas
    context_lines = []
    for doc in search_results:
        line = f"- {doc['datetime']}: {doc['title']}"
        if doc.get('notes'):
            line += f" ({doc['notes']})"
        context_lines.append(line)
    context = "\n".join(context_lines)

    prompt = prompt_template.format(question=query, context=context)
    return prompt


In [ ]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GEMINI_API_KEY}"

payload = {
    "contents": [{"parts": [{"text": prompt}]}],
}
response = requests.post(url, json=payload)
content = response.json()["candidates"][0]["content"]["parts"][0]["text"]
print(content)